# SQL 的五十道練習

> 條件邏輯

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD ../databases/imdb.db

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/twElection2020.db" AS twElection2020;

In [4]:
ATTACH "../databases/covid19.db" AS covid19;

## 以 `CASE` 敘述衍生欄位

## 條件除了能夠運用在 `WHERE` 敘述篩選資料，另外一個常見的應用場景是在 `CASE` 敘述來衍生欄位。

```sql
-- CASE 敘述
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias
  FROM table_name;
```

In [5]:
-- 將上映日期分為兩類的 CASE 敘述
SELECT title,
       release_year,
       CASE WHEN release_year >= 2000 THEN 1
            ELSE 0 END AS is_after_millennium
  FROM top_rated_movies
 LIMIT 10;

title,release_year,is_after_millennium
The Shawshank Redemption,1994,0
The Godfather,1972,0
The Godfather: Part II,1974,0
The Dark Knight,2008,1
12 Angry Men,1957,0
Schindler's List,1993,0
The Lord of the Rings: The Return of the King,2003,1
Pulp Fiction,1994,0
"The Good, the Bad and the Ugly",1966,0
The Lord of the Rings: The Fellowship of the Ring,2001,1


In [6]:
-- 將位置分為三類的 CASE 敘述
SELECT firstName || ' ' || lastName AS player_name,
       pos,
       CASE WHEN pos IN ('C', 'C-F') THEN 'Center'
            WHEN pos IN ('G', 'G-F') THEN 'Guard'
            ELSE 'Forward' END AS pos_new
  FROM players
 LIMIT 10;

player_name,pos,pos_new
LeBron James,F,Forward
Carmelo Anthony,F,Forward
Udonis Haslem,F,Forward
Dwight Howard,C-F,Center
Andre Iguodala,G-F,Guard
Trevor Ariza,F,Forward
Chris Paul,G,Guard
Lou Williams,G,Guard
LaMarcus Aldridge,C-F,Center
Rudy Gay,F-G,Forward


## 當條件有交集的時候，撰寫 `CASE` 敘述要特別注意順序

In [7]:
-- heightMeters > 1.90 與 heightMeters > 2.10 條件交集
SELECT DISTINCT CASE WHEN heightMeters > 2.10 THEN 'Tall'
                WHEN heightMeters > 1.90 THEN 'Medium'
                ELSE 'Short' END AS height_category
  FROM players;

height_category
Medium
Short
Tall


In [8]:
-- heightMeters > 1.90 與 heightMeters > 2.10 條件交集
SELECT DISTINCT CASE WHEN heightMeters > 1.90 THEN 'Medium'
                WHEN heightMeters > 2.10 THEN 'Tall'
                ELSE 'Short' END AS height_category
  FROM players;

height_category
Medium
Short


##  如果不想特別注意順序，亦可以將條件設計為互斥

In [9]:
SELECT DISTINCT CASE WHEN heightMeters > 1.90 AND 
                          heightMeters <= 2.10 THEN 'Medium'
                     WHEN heightMeters > 2.10 THEN 'Tall'
                     ELSE 'Short' END AS height_category
  FROM players;

height_category
Medium
Short
Tall


## 以 `CASE` 敘述衍生的欄位排序查詢結果

## 除了能夠在 `SELECT` 加入 `CASE` 敘述，亦能夠在 `ORDER BY` 敘述加入。

```sql
-- 以 CASE 敘述衍生的欄位排序查詢結果
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias
  FROM table_name
 ORDER BY alias ASC|DESC;
```

In [10]:
-- 以 CASE 敘述衍生的欄位排序查詢結果
SELECT firstName || ' ' || lastName AS player_name,
       CASE WHEN pos IN ('C', 'C-F') THEN 'Center'
            WHEN pos IN ('G', 'G-F') THEN 'Guard'
            ELSE 'Forward' END AS pos_new
  FROM players
 ORDER BY pos_new
 LIMIT 10;

player_name,pos_new
Dwight Howard,Center
LaMarcus Aldridge,Center
Al Horford,Center
Marc Gasol,Center
Brook Lopez,Center
Robin Lopez,Center
JaVale McGee,Center
DeAndre Jordan,Center
DeMarcus Cousins,Center
Ed Davis,Center


## 也能僅在 `ORDER BY` 敘述中加入 `CASE`，不用先加 `SELECT`

In [11]:
-- 以 CASE 敘述衍生的欄位排序查詢結果
SELECT firstName || ' ' || lastName AS player_name
  FROM players
 ORDER BY (CASE WHEN pos IN ('C', 'C-F') THEN 'Center'
                WHEN pos IN ('G', 'G-F') THEN 'Guard'
                ELSE 'Forward' END)
 LIMIT 10;

player_name
Dwight Howard
LaMarcus Aldridge
Al Horford
Marc Gasol
Brook Lopez
Robin Lopez
JaVale McGee
DeAndre Jordan
DeMarcus Cousins
Ed Davis


## 以 `CASE` 敘述衍生的欄位篩選資料

## 除了能夠在 `SELECT` 以及 `ORDER BY` 加入 `CASE` 敘述，亦能夠在 `WHERE` 敘述加入。

```sql
-- 以 CASE 敘述衍生的欄位篩選資料
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias
  FROM table_name
 WHERE column_name COMPARISON_OPERATOR alias;
```

In [12]:
-- 以 CASE 敘述衍生的欄位篩選資料
SELECT firstName || ' ' || lastName AS player_name,
       CASE WHEN pos IN ('C', 'C-F') THEN 'Center'
            WHEN pos IN ('G', 'G-F') THEN 'Guard'
            ELSE 'Forward' END AS pos_new
  FROM players
 WHERE pos_new = 'Center'
 LIMIT 10;

player_name,pos_new
Dwight Howard,Center
LaMarcus Aldridge,Center
Al Horford,Center
Marc Gasol,Center
Brook Lopez,Center
Robin Lopez,Center
JaVale McGee,Center
DeAndre Jordan,Center
DeMarcus Cousins,Center
Ed Davis,Center


## 重點統整

- 條件除了能夠運用在 `WHERE` 敘述篩選資料，另外一個常見的應用場景是在 `CASE` 敘述來衍生欄位。
- 除了能夠在 `SELECT` 加入 `CASE` 敘述，亦能夠在 `ORDER BY` 敘述與 `WHERE` 敘述加入。

## 目前我們會的 SQL

```sql
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias
  FROM table_name
 WHERE condition
 ORDER BY column_name
 LIMIT m;
```